In [2]:
import pandas as pd
import os
from Bio import Entrez, SeqIO
import tqdm
from collections import Counter
import numpy as np
from scipy.stats import multinomial
import numpy as np
import matplotlib.pyplot as plt


In [3]:
iterator = pd.read_csv("/Users/reem/Downloads/metadata_tsv_2025_09_28/metadata.tsv", sep="\t", chunksize=1000)
metadata_df = pd.concat([chunk for chunk in tqdm.tqdm(iterator, desc='Loading data')])



Loading data: 17454it [01:28, 198.06it/s]


In [4]:
metadata_df.head()

,Virus name,Last vaccinated,Passage details/history,Type,Accession ID,Collection date,Location,Additional location information,Sequence length,Host,...,Pango version,Variant,AA Substitutions,Submission date,Is reference?,Is complete?,Is high coverage?,Is low coverage?,N-Content,GC-Content
0,hCoV-19/Israel/SMC-7103248/2022,NaN,Original,betacoronavirus,EPI_ISL_15173403,2022-09-04,Asia / Israel,NaN,29425,Human,...,consensus call,Former VOC Omicron (B.1.1.529+BA.*),"(NSP5_P132H,Spike_H69del,NSP1_M85del,NSP3_P679...",2022-09-29,NaN,True,NaN,True,0.192795,0.380052
1,hCoV-19/Israel/ICH-741216991/2022,NaN,Original,betacoronavirus,EPI_ISL_12375715,2022-04-09,Asia / Israel,NaN,29442,Human,...,consensus call,Former VOC Omicron (B.1.1.529+BA.*),"(NSP5_P132H,NSP3_G489S,Spike_L24del,NSP4_T327I...",2022-04-28,NaN,True,NaN,NaN,0.031520,0.377744
2,hCoV-19/South Korea/KDCA278478/2023,NaN,Original,betacoronavirus,EPI_ISL_18712060,2023-11-28,Asia / South Korea,NaN,29752,Human,...,consensus call,Former VOI (EG.5+EG.5.*),"(NSP5_P132H,Spike_Q52H,NSP2_A510V,NSP12_G671S,...",2024-01-03,NaN,True,NaN,NaN,0.001916,0.378818
3,hCoV-19/USA/CA-CDC-FG-155526/2021,NaN,Original,betacoronavirus,EPI_ISL_5901503,2021-10-23,North America / USA / California,NaN,29812,Human,...,consensus call,Former VOC Delta (B.1.617.2+AY.*),"(NSP3_A480V,Spike_I1114V,Spike_V1264L,N_D63G,N...",2021-11-03,NaN,True,True,NaN,0.000101,0.379617
4,hCoV-19/USA/CA-CDPH-3000143864/2021,NaN,Original,betacoronavirus,EPI_ISL_5106195,2021-08-06,North America / USA / California / Contra Cost...,NaN,29874,Human,...,consensus call,Former VOC Delta (B.1.617.2+AY.*),"(N_G215C,N_D63G,N_R203M,NSP12_G671S,NSP16_Q238...",2021-10-13,NaN,True,NaN,NaN,0.020252,0.379664


In [5]:
metadata_df['country'] = metadata_df['Virus name'].str.split("/").str[1]

In [6]:
countries = ['Australia','Italy','Japan','Israel','South Korea', 'United Kingdom','Germany','Spain','USA','Denmark','Belgium','India','Brazil']
start_date = '2021-11-04'
end_date = '2025-09-01'

metadata_df['label'] = (metadata_df['country'].isin(countries) & (metadata_df['Collection date'] >= start_date)
& (metadata_df['Collection date'] <= end_date))

print(metadata_df['label'].head())
metadata_df['label'].value_counts()

    

0     True
1     True
2     True
3    False
4    False
Name: label, dtype: bool


label
False    11166614
True      6286855
Name: count, dtype: int64

In [7]:
metadata_df.head()

,Virus name,Last vaccinated,Passage details/history,Type,Accession ID,Collection date,Location,Additional location information,Sequence length,Host,...,AA Substitutions,Submission date,Is reference?,Is complete?,Is high coverage?,Is low coverage?,N-Content,GC-Content,country,label
0,hCoV-19/Israel/SMC-7103248/2022,NaN,Original,betacoronavirus,EPI_ISL_15173403,2022-09-04,Asia / Israel,NaN,29425,Human,...,"(NSP5_P132H,Spike_H69del,NSP1_M85del,NSP3_P679...",2022-09-29,NaN,True,NaN,True,0.192795,0.380052,Israel,True
1,hCoV-19/Israel/ICH-741216991/2022,NaN,Original,betacoronavirus,EPI_ISL_12375715,2022-04-09,Asia / Israel,NaN,29442,Human,...,"(NSP5_P132H,NSP3_G489S,Spike_L24del,NSP4_T327I...",2022-04-28,NaN,True,NaN,NaN,0.031520,0.377744,Israel,True
2,hCoV-19/South Korea/KDCA278478/2023,NaN,Original,betacoronavirus,EPI_ISL_18712060,2023-11-28,Asia / South Korea,NaN,29752,Human,...,"(NSP5_P132H,Spike_Q52H,NSP2_A510V,NSP12_G671S,...",2024-01-03,NaN,True,NaN,NaN,0.001916,0.378818,South Korea,True
3,hCoV-19/USA/CA-CDC-FG-155526/2021,NaN,Original,betacoronavirus,EPI_ISL_5901503,2021-10-23,North America / USA / California,NaN,29812,Human,...,"(NSP3_A480V,Spike_I1114V,Spike_V1264L,N_D63G,N...",2021-11-03,NaN,True,True,NaN,0.000101,0.379617,USA,False
4,hCoV-19/USA/CA-CDPH-3000143864/2021,NaN,Original,betacoronavirus,EPI_ISL_5106195,2021-08-06,North America / USA / California / Contra Cost...,NaN,29874,Human,...,"(N_G215C,N_D63G,N_R203M,NSP12_G671S,NSP16_Q238...",2021-10-13,NaN,True,NaN,NaN,0.020252,0.379664,USA,False


In [8]:
metadata_df.to_csv("/Users/reem/Mov/metadata_labeled.tsv",sep="\t")